# 🍏 Health Resource Search Agent Tutorial 🍎

Welcome to the **Health Resource Search Agent** tutorial! We'll use **Azure AI Foundry** SDKs to build an assistant that can:

1. **Upload** health and recipe files into a vector store.
2. **Create an Agent** with a **File Search** tool.
3. **Search** these documents for relevant dietary info.
4. **Answer** health and wellness questions (with disclaimers!).

### ⚠️ Important Medical Disclaimer ⚠️
> **All health information in this notebook is for general educational purposes only and is not a substitute for professional medical advice, diagnosis, or treatment.** Always seek the advice of a qualified healthcare professional with any questions you may have.

## Prerequisites
- Complete Agent basics notebook - [1-basics.ipynb](1-basics.ipynb)
- **Roles**  
  1. **Azure AI Developer** on your Azure AI Foundry project.
  2. **Storage Blob Data Contributor** on the project’s Storage account.
  3. If standard agent setup is used with your own Search resource, also ensure you have **Cognitive Search Data Contributor** on that resource.

## Let's Get Searching!
We'll show you how to upload some sample files, create a vector store for them, then spin up an agent that can search these resources for dietary guidelines, recipes, and more. Enjoy!

<img src="./seq-diagrams/3-file-search.png" width="30%"/>


## 🔐 Authentication Setup

Before running the next cell, make sure you're authenticated with Azure CLI. Run this command in your terminal:

```bash
az login --use-device-code
```

This will provide you with a device code and URL to authenticate in your browser, which is useful for:
- Remote development environments
- Systems without a default browser
- Corporate environments with strict security policies

After successful authentication, you can proceed with the notebook cells below.

## 1. Initial Setup
Here we import needed libraries, load environment variables from `.env`, and initialize our **AIProjectClient**. Let's do this! 🎉

In [ ]:
import os
import time
from pathlib import Path

from dotenv import load_dotenv
from azure.identity import InteractiveBrowserCredential
from azure.ai.projects import AIProjectClient

# Load environment variables from parent .env
notebook_path = Path().absolute()
parent_dir = notebook_path.parent
load_dotenv(parent_dir.parent / '.env')

# Get tenant ID and connection string
tenant_id = os.environ.get("TENANT_ID")
ai_foundry_project_endpoint = os.environ.get("AI_FOUNDRY_PROJECT_ENDPOINT")

print(f"🔑 Using Tenant ID: {tenant_id}")

# Initialize AIProjectClient with simplified browser-based authentication
try:
    print("🌐 Using browser-based authentication to bypass Azure CLI cache issues...")
    
    # Use only InteractiveBrowserCredential with the specific tenant
    credential = InteractiveBrowserCredential(tenant_id=tenant_id)
    
    # Create the project client using endpoint (ai_foundry_project_endpoint is actually the endpoint URL)
    project_client = AIProjectClient(
        endpoint=ai_foundry_project_endpoint,
        credential=credential
    )
    print("✅ Successfully initialized AIProjectClient")
    
except Exception as e:
    print(f"❌ Error initializing project client: {e}")
    print("💡 Please complete the browser authentication prompt that should appear")

## 2. Prepare Sample Files 🍲🗒
We'll create some dummy .md files (for recipes and guidelines). Then we'll store them in a vector store for searching.


In [ ]:
def create_sample_files():
    recipes_md = (
        """# Healthy Recipes Database\n\n"
        "## Gluten-Free Recipes\n"
        "1. Quinoa Bowl\n"
        "   - Ingredients: quinoa, vegetables, olive oil\n"
        "   - Instructions: Cook quinoa, add vegetables\n\n"
        "2. Rice Pasta with Vegetables\n"
        "   - Ingredients: rice pasta, mixed vegetables\n"
        "   - Instructions: Boil pasta, sauté vegetables\n\n"
        "## Diabetic-Friendly Recipes\n"
        "1. Low-Carb Stir Fry\n"
        "   - Ingredients: chicken, vegetables, tamari sauce\n"
        "   - Instructions: Cook chicken, add vegetables\n\n"
        "2. Greek Salad\n"
        "   - Ingredients: cucumber, tomatoes, feta, olives\n"
        "   - Instructions: Chop vegetables, combine\n\n"
        "## Heart-Healthy Recipes\n"
        "1. Baked Salmon\n"
        "   - Ingredients: salmon, lemon, herbs\n"
        "   - Instructions: Season salmon, bake\n\n"
        "2. Mediterranean Bowl\n"
        "   - Ingredients: chickpeas, vegetables, tahini\n"
        "   - Instructions: Combine ingredients\n"""
    )

    guidelines_md = (
        """# Dietary Guidelines\n\n"
        "## General Guidelines\n"
        "- Eat a variety of foods\n"
        "- Control portion sizes\n"
        "- Stay hydrated\n\n"
        "## Special Diets\n"
        "1. Gluten-Free Diet\n"
        "   - Avoid wheat, barley, rye\n"
        "   - Focus on naturally gluten-free foods\n\n"
        "2. Diabetic Diet\n"
        "   - Monitor carbohydrate intake\n"
        "   - Choose low glycemic foods\n\n"
        "3. Heart-Healthy Diet\n"
        "   - Limit saturated fats\n"
        "   - Choose lean proteins\n"""
    )

    # Save to local files using configurable filenames
    recipes_filename = os.environ.get("RECIPES_FILENAME", "recipes.md")
    guidelines_filename = os.environ.get("GUIDELINES_FILENAME", "guidelines.md")
    
    with open(recipes_filename, "w", encoding="utf-8") as f:
        f.write(recipes_md)
    with open(guidelines_filename, "w", encoding="utf-8") as f:
        f.write(guidelines_md)

    print(f"📄 Created sample resource files: {recipes_filename}, {guidelines_filename}")
    return [recipes_filename, guidelines_filename]

sample_files = create_sample_files()

#### ✨ Note on Search Permissions
When creating the vector store, you must also have **Cognitive Search Data Contributor** role on your Azure AI Search resource (if you're using the standard agent setup with your own Search resource). Missing this role will often cause a **Forbidden** error. See [Authentication Setup](../../1-introduction/1-authentication.ipynb#4-add-agent-service-permissions) for details on configuring permissions.


## 3. Create a Vector Store 📚
We'll upload our newly created files and group them into a single vector store for searching. This is how the agent can later find relevant text.

In [ ]:
def create_vector_store(files, store_name="my_health_resources"):
    try:
        # Step 1: Upload files to Azure AI Agent service
        # Each file needs to be uploaded individually and we'll collect their IDs
        uploaded_ids = []
        for fp in files:
            # upload_and_poll ensures the upload is complete before continuing
            # "assistants" purpose tells the service these files are for agent usage
            upl = project_client.agents.files.upload_and_poll(
                file_path=fp,
                purpose="assistants"
            )
            uploaded_ids.append(upl.id)
            print(f"✅ Uploaded: {fp} -> File ID: {upl.id}")

        # Step 2: Create a vector store from the uploaded files
        # A vector store converts text into numerical vectors for semantic search
        # create_and_poll waits until indexing is complete
        vs = project_client.agents.vector_stores.create_and_poll(
            file_ids=uploaded_ids,  # Pass all our uploaded file IDs
            name=store_name         # Give our vector store a friendly name
        )
        print(f"🎉 Created vector store '{store_name}', ID: {vs.id}")
        return vs, uploaded_ids
    except Exception as e:
        print(f"❌ Error creating vector store: {e}")
        # Let's see what methods are actually available
        print("Available files methods:")
        for method in dir(project_client.agents.files):
            if not method.startswith('_'):
                print(f"  - {method}")
        print("Available vector_stores methods:")
        for method in dir(project_client.agents.vector_stores):
            if not method.startswith('_'):
                print(f"  - {method}")
        return None, []

# Initialize empty variables to store our vector store and file IDs
vector_store, file_ids = None, []

# If we successfully created sample files earlier, create a vector store from them
if sample_files:
    vector_store, file_ids = create_vector_store(sample_files, "health_resources_example")
else:
    print("⚠️ No sample files available - please run the previous cell to create sample files first")

## 4. Create the Health Resource Agent 🔎
We use a **FileSearchTool** pointing to our newly created vector store, then create the Agent with instructions about disclaimers, dietary help, etc.

In [ ]:
def create_health_resource_agent(vstore_id):
    try:
        # Create an AI agent that will use file search and follow specific instructions
        # Based on the code interpreter pattern, let's create a simple agent first
        agent = project_client.agents.create_agent(
            model=os.environ.get("MODEL_DEPLOYMENT_NAME", "gpt-4o-mini"),
            name="health-search-agent",
            instructions="""
                You are a health resource advisor with access to dietary and recipe files.
                You:
                1. Always present disclaimers (you're not a doctor!)
                2. Provide references to the files when possible
                3. Focus on general nutrition or recipe tips.
                4. Encourage professional consultation for more detailed advice.
            """,
            tools=[{"type": "file_search"}]
        )
        print(f"🎉 Created health resource agent, ID: {agent.id}")
        print("📋 Vector store will be attached at message level for better compatibility")
        return agent
    except Exception as e:
        print(f"❌ Error creating health resource agent: {e}")
        return None

# Ensure the placeholder variable exists to avoid NameError on first run
if 'health_agent' not in globals():
    health_agent = None

# Let's recreate the agent with the clean approach
if vector_store:
    # Delete the old agent first (only if it exists already)
    if health_agent:
        try:
            project_client.agents.delete_agent(health_agent.id)
            print("🗑️ Deleted old agent")
        except Exception as delete_err:
            # Swallow errors but surface minimal context for debugging
            print(f"⚠️ Could not delete previous agent: {delete_err}")
    
    # Create new agent
    health_agent = create_health_resource_agent(vector_store.id)

## 5. Searching Health Resources 🏋️👩‍🍳
We'll create a new conversation thread and ask queries like “Gluten-free recipe ideas?” or “Heart-healthy meal plan?” The agent will do file search on the vector store to find relevant info.

In [ ]:
def create_search_thread(agent):
    try:
        # In Azure AI Agent service, conversations happen in "threads"
        # A thread maintains the context and history of a conversation
        # Here we create a new empty thread to start a fresh conversation
        thread = project_client.agents.threads.create()
        print(f"📝 Created new search thread, ID: {thread.id}")
        return thread
    except Exception as e:
        print(f"❌ Error creating search thread: {e}")
        return None

def ask_search_question_with_files(thread_id, agent_id, user_question, file_ids_list):
    try:
        # Create message with file attachments for search
        # Based on the code interpreter pattern, attach files at message level
        attachments = []
        for file_id in file_ids_list:
            attachments.append({
                "file_id": file_id,
                "tools": [{"type": "file_search"}]
            })
            
        message = project_client.agents.messages.create(
            thread_id=thread_id,
            role="user",
            content=user_question,
            attachments=attachments
        )
        print(f"🔎 Searching with {len(file_ids_list)} files: '{user_question}'")

        # Create and process the run
        run = project_client.agents.runs.create_and_process(
            thread_id=thread_id,
            agent_id=agent_id
        )
        print(f"🤖 Run finished with status: {run.status}")
        if run.last_error:
            print(f"Error details: {run.last_error}")
        return run
    except Exception as e:
        print(f"❌ Error with file search: {e}")
        return None

def ask_simple_question(thread_id, agent_id, user_question):
    try:
        message = project_client.agents.messages.create(
            thread_id=thread_id,
            role="user",
            content=user_question
        )
        run = project_client.agents.runs.create_and_process(
            thread_id=thread_id,
            agent_id=agent_id
        )
        print(f"🤖 Simple run finished with status: {run.status}")
        return run
    except Exception as e:
        print(f"❌ Error with simple question: {e}")
        return None

# Now let's test our search functionality!
# Initialize search_thread as None to avoid NameError
search_thread = None

# First check if we have our health agent available
if health_agent:
    # Create a new conversation thread
    search_thread = create_search_thread(health_agent)

    if search_thread:
        # Define some test questions that demonstrate different types of health queries
        # The agent will search our uploaded health documents to answer these
        queries = [
            "Could you suggest a gluten-free lunch recipe?",
            "Show me some heart-healthy meal ideas.",
            "What guidelines do you have for someone with diabetes?"
        ]

        # Process each query one at a time using file attachments
        # The agent will have access to our uploaded files to search through
        for q in queries:
            if file_ids:
                ask_search_question_with_files(search_thread.id, health_agent.id, q, file_ids)
            else:
                ask_simple_question(search_thread.id, health_agent.id, q)

## 6. View Results & Citations 📄
We'll read the conversation thread to see how the agent responded and see if it cited the correct files.

In [ ]:
def display_thread_messages(thread_id):
    try:
        # Retrieve all messages in this conversation thread using the Azure AI Agent SDK
        # Messages contain the back-and-forth between user and AI agent
        messages = project_client.agents.messages.list(thread_id=thread_id)

        # Display the conversation history
        print("\n🗣️ Conversation so far:")
        message_list = list(messages)  # Convert ItemPaged to list
        
        for m in reversed(message_list):
            # Each message may have multiple content pieces
            # We're interested in the text content (vs other types like images)
            if m.content:
                last_content = m.content[-1]
                if hasattr(last_content, "text"):
                    # Print who said what (ASSISTANT or USER) along with their message
                    text_value = last_content.text.value
                    print(f"{m.role.upper()}: {text_value}\n" + "-"*50 + "\n")

        # The agent can cite specific passages from the uploaded documents
        # Let's check if it referenced any files in its responses
        print("\n📎 Checking for citations...")
        citation_count = 0
        for m in message_list:
            if m.content:
                for content_item in m.content:
                    if hasattr(content_item, "text") and hasattr(content_item.text, "annotations"):
                        for annotation in content_item.text.annotations:
                            citation_count += 1
                            if hasattr(annotation, "file_citation"):
                                print(f"- Citation {citation_count}: '{annotation.text}' from file ID: {annotation.file_citation.file_id}")
                            else:
                                print(f"- Citation {citation_count}: '{annotation.text}'")
        
        if citation_count == 0:
            print("No citations found - checking if agent used our uploaded files...")
            # Check if any responses mention content from our files
            for m in message_list:
                if m.role == "assistant" and m.content:
                    for content_item in m.content:
                        if hasattr(content_item, "text"):
                            text = content_item.text.value.lower()
                            if any(keyword in text for keyword in ['quinoa', 'salmon', 'gluten-free', 'diabetic', 'heart-healthy']):
                                print(f"✅ Agent appears to be using file content (found relevant keywords)")
                                break

    except Exception as e:
        # Gracefully handle any errors that might occur when displaying messages
        print(f"❌ Error displaying messages: {e}")
        import traceback
        traceback.print_exc()

# Display the conversation history for our search thread
# Check if search_thread exists and was created successfully before using it
if 'search_thread' in locals() and search_thread:
    display_thread_messages(search_thread.id)
else:
    print("⚠️ No search thread available to display.")
    print("💡 Please run the previous cell first to create a search thread and ask questions.")
    print("🔄 Make sure the health agent was created successfully before proceeding.")

## 7. Cleanup & Best Practices 🧹
We'll optionally remove the vector store, the uploaded files, and the agent. In a production environment, you might keep them around longer. Meanwhile, here are some tips:

1. **Resource Management**
   - Keep files grouped by category, regularly prune old or irrelevant files.
   - Clear out test agents or vector stores once you're done.

2. **Search Queries**
   - Provide precise or multi-part queries.
   - Consider synonyms or alternative keywords ("gluten-free" vs "celiac").
   
3. **Health Information**
   - Always disclaim that you are not a medical professional.
   - Encourage users to see doctors for specific diagnoses.

4. **Performance**
   - Keep an eye on vector store size.
   - Evaluate search accuracy with `azure-ai-evaluation`!


In [ ]:
def cleanup_all():
    try:
        # Check if we have a vector store and delete it
        # Vector stores are where we store the embeddings (numerical representations) 
        # of our documents for semantic search
        if 'vector_store' in globals() and vector_store:
            project_client.agents.vector_stores.delete(vector_store.id)
            print("🗑️ Deleted vector store.")

        # Remove any files we uploaded to Azure AI Search
        # These were the documents our agent used as its knowledge base
        if 'file_ids' in globals() and file_ids:
            for fid in file_ids:
                project_client.agents.files.delete(fid)
            print("🗑️ Deleted uploaded files from the service.")

        # Delete the AI agent we created
        # This frees up resources since we're done with our demo
        if 'health_agent' in globals() and health_agent:
            project_client.agents.delete_agent(health_agent.id)
            print("🗑️ Deleted health resource agent.")

        # Clean up any local files we created during the demo
        # This keeps our workspace tidy
        if 'sample_files' in globals() and sample_files:
            for sf in sample_files:
                if os.path.exists(sf):
                    os.remove(sf)
            print("🗑️ Deleted local sample files.")

    except Exception as e:
        # If anything goes wrong during cleanup, we'll see what happened
        print(f"❌ Error during cleanup: {e}")

# Run our cleanup function to remove all resources we created
# This is good practice in a tutorial/demo environment
cleanup_all()

# Congratulations! 🎉

You've successfully completed the **Health Resource Search Agent** tutorial! Here's what was accomplished:

## ✅ **What We Built**

### **🔍 File Search Agent**
- Created an AI agent with **file search** capabilities
- Enabled the agent to search through uploaded health documents using semantic search
- Configured health-focused instructions with appropriate medical disclaimers

### **📚 Key Features Demonstrated**

1. **📄 File Upload & Vector Store Creation**
   - Uploaded health resource files to Azure AI service
   - Created a vector store for semantic document search
   - **Fixed file upload methods** by using the correct API patterns

2. **🔎 Semantic Document Search**
   - Agent successfully searched through health and recipe documents
   - Provided relevant answers based on uploaded file contents
   - **Fixed file search access** by using message-level file attachments

3. **🏥 Health-Focused Responses**
   - Agent answered questions about gluten-free recipes, heart-healthy meals, and diabetic guidelines
   - Provided responsible AI disclaimers about medical advice
   - Referenced content from uploaded documents

4. **🧹 Resource Management**
   - Properly cleaned up vector stores, files, and agents
   - Demonstrated best practices for resource lifecycle management

## 🛠️ **Important Fixes Applied**

### **1. Import Issues Fixed**
```python
# ❌ This didn't work:
from azure.ai.projects.models import FileSearchTool, FilePurpose

# ✅ This approach works:
# Create tools directly as dictionaries: {"type": "file_search"}
# Use string values like "assistants" instead of FilePurpose enum
```

### **2. Vector Store Creation Fixed**
```python
# ❌ Original broken approach:
project_client.agents.upload_file_and_poll()
project_client.agents.create_vector_store_and_poll()

# ✅ Working API structure:
project_client.agents.files.upload_and_poll()
project_client.agents.vector_stores.create_and_poll()
```

### **3. File Search Access Fixed**
The key breakthrough was using **message-level file attachments** instead of agent-level tool_resources:

```python
# ❌ This caused SDK compatibility issues:
agent = project_client.agents.create_agent(
    tools=[{"type": "file_search"}],
    tool_resources={"file_search": {"vector_store_ids": [vs_id]}}
)

# ✅ This works reliably:
# 1. Create simple agent with file_search tool
agent = project_client.agents.create_agent(
    tools=[{"type": "file_search"}]
)

# 2. Attach files at message level
message = project_client.agents.messages.create(
    thread_id=thread_id,
    role="user",
    content=question,
    attachments=[{
        "file_id": file_id,
        "tools": [{"type": "file_search"}]
    }]
)
```

### **4. API Method Updates**
```python
# ❌ Old patterns:
project_client.agents.create_thread()
project_client.agents.create_message()
project_client.agents.create_and_process_run()
project_client.agents.list_messages()

# ✅ Correct API structure:
project_client.agents.threads.create()
project_client.agents.messages.create()
project_client.agents.runs.create_and_process()
project_client.agents.messages.list()
```

## 🎯 **Key Concepts Learned**

- **File Search Tool**: Enables agents to search through uploaded documents using semantic search
- **Vector Stores**: Convert documents into searchable numerical vectors
- **Message-Level Attachments**: More reliable than agent-level tool_resources for current SDK version
- **Semantic Search**: Agents can find relevant content even when exact words don't match
- **Responsible AI**: Always include medical disclaimers for health-related content

## 🔍 **API Methods Mastered**

- `project_client.agents.files.upload_and_poll()` - File upload with polling
- `project_client.agents.vector_stores.create_and_poll()` - Vector store creation
- `project_client.agents.create_agent()` - Agent creation with file search capabilities
- `project_client.agents.threads.create()` - Conversation thread creation
- `project_client.agents.messages.create()` - Creating messages with file attachments
- `project_client.agents.runs.create_and_process()` - Running agents with file search
- `project_client.agents.messages.list()` - Retrieving conversation history

## 🚀 **What's Next?**

Continue your Azure AI Agent Service journey with these advanced topics:

- **[4-bing_grounding.ipynb](4-bing_grounding.ipynb)** - Agents with real-time web search capabilities  
- **[5-agents-aisearch.ipynb](5-agents-aisearch.ipynb)** - Integration with Azure AI Search for enterprise knowledge
- **[6-agents-az-functions.ipynb](6-agents-az-functions.ipynb)** - Agents that can trigger Azure Functions and workflows

## 💡 **Best Practices Recap**

1. **File Attachment Strategy** - Use message-level file attachments instead of agent-level tool_resources
2. **API Structure** - Use the nested client structure (agents.files, agents.threads, etc.)
3. **Vector Store Management** - Keep documents organized and clean up when done
4. **Health Content** - Always include medical disclaimers for health-related responses
5. **Resource Cleanup** - Delete agents, vector stores, and files to manage costs
6. **Error Handling** - Implement fallback approaches for SDK compatibility issues
7. **Semantic Search** - Leverage vector stores for intelligent document search

## 🔧 **Troubleshooting Guide**

**If file search isn't working:**
1. ✅ Use message-level file attachments instead of agent-level tool_resources
2. ✅ Verify files were uploaded successfully (check file IDs)
3. ✅ Use correct API structure (agents.files.upload_and_poll)
4. ✅ Attach individual files to messages, not just vector stores

**If import errors occur:**
1. ✅ Use dictionary-based tool definitions: `{"type": "file_search"}`
2. ✅ Use string literals instead of imported enums: `"assistants"`
3. ✅ Import only what's actually available in the SDK version

Ready to explore more advanced file search and integration patterns? **Let's continue!** 🚀

---

*Happy (healthy) searching!* 🔍💚